# 🎨 Embeddings - Représentations Vectorielles du Langage

## 🎯 Objectifs

Dans ce notebook, nous allons découvrir :

- 🔢 **Le problème des IDs** - Pourquoi les nombres ne suffisent pas
- 🎨 **Word Embeddings** - Représentations vectorielles denses
- 📐 **Similarité sémantique** - Mots similaires = vecteurs proches
- 🧮 **Implémentation** - Embedding layer from scratch
- 🔍 **Word2Vec concepts** - Skip-gram et CBOW

---

## 🤔 Le Problème des IDs Bruts

### Rappel : Tokenization

Dans le notebook précédent, on a transformé du texte en IDs :

```
"Le chat mange" → [145, 298, 1023]
```

### Problème : Les IDs ne capturent PAS la sémantique

```python
# Ces IDs sont arbitraires
"chat"  → ID 145
"chien" → ID 298
"souris" → ID 1023

# Questions :
# - "chat" et "chien" sont similaires (animaux domestiques)
# - Mais 145 et 298 sont aussi éloignés que 145 et 1023 !
# - Comment capturer cette similarité ?
```

### Solution : Embeddings !

Transformer chaque ID en un **vecteur dense** qui capture le sens :

```python
"chat"  → [0.2, -0.5, 0.8, 0.1, ...]  # 256 dimensions
"chien" → [0.3, -0.4, 0.7, 0.2, ...]  # Proche de "chat" !
"souris" → [-0.8, 0.9, -0.3, 0.5, ...] # Différent
```

---

In [ ]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

np.random.seed(42)
print("✓ Prêt pour les embeddings !")

## 🎨 Qu'est-ce qu'un Embedding ?

### Définition

Un **embedding** est une représentation vectorielle **dense** et **continue** d'un token.

### Comparaison : One-Hot vs Embedding

#### 1️⃣ One-Hot Encoding (Naïf)

```python
vocab_size = 50000

"chat" (ID 145) → [0, 0, 0, ..., 1, ..., 0]  # 1 à la position 145
#                  ↑ 50,000 dimensions !
```

**Problèmes** :
- ❌ Vecteurs très grands (vocab_size dimensions)
- ❌ Sparse (que des 0 et un seul 1)
- ❌ Aucune similarité (tous orthogonaux)
- ❌ Énormément de paramètres

#### 2️⃣ Dense Embedding (Moderne) ✅

```python
embedding_dim = 256

"chat" (ID 145) → [0.23, -0.45, 0.67, ..., 0.12]  # 256 dimensions
#                  ↑ Dense (toutes les valeurs utilisées)
```

**Avantages** :
- ✅ Compact (256 vs 50,000 dimensions)
- ✅ Dense (toutes les dimensions informatives)
- ✅ Capture la similarité
- ✅ Appris automatiquement

---

In [ ]:
# Démonstration : One-Hot vs Embedding

vocab_size = 10
embedding_dim = 4

# One-Hot
def one_hot(token_id, vocab_size):
    vec = np.zeros(vocab_size)
    vec[token_id] = 1
    return vec

# Dense Embedding (aléatoire pour l'exemple)
embedding_matrix = np.random.randn(vocab_size, embedding_dim) * 0.1

def embed(token_id):
    return embedding_matrix[token_id]

# Comparer
token_id = 3

print("🔢 One-Hot Encoding:")
oh = one_hot(token_id, vocab_size)
print(f"   Token {token_id}: {oh}")
print(f"   Taille: {len(oh)} dimensions")
print(f"   Sparse: {(oh == 0).sum()} zéros / {len(oh)}\n")

print("🎨 Dense Embedding:")
emb = embed(token_id)
print(f"   Token {token_id}: {emb}")
print(f"   Taille: {len(emb)} dimensions")
print(f"   Dense: Toutes les valeurs utilisées")

print(f"\n💾 Économie de mémoire: {vocab_size / embedding_dim:.1f}× plus compact !")

---

## 📐 Similarité Sémantique

### Concept Clé

**Mots similaires → Vecteurs proches dans l'espace**

### Mesure : Cosine Similarity

```
similarity(A, B) = (A · B) / (||A|| × ||B||)
```

- Valeur entre -1 et 1
- 1 = Très similaires
- 0 = Orthogonaux (pas de relation)
- -1 = Opposés

### Exemples Attendus

```python
similarity("roi", "reine")     ≈ 0.85  # Très similaires
similarity("roi", "homme")     ≈ 0.45  # Liés
similarity("roi", "pomme")     ≈ 0.05  # Pas liés
```

---

In [ ]:
def cosine_similarity(a, b):
    """
    Calcule la similarité cosinus entre deux vecteurs
    """
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    
    return dot_product / (norm_a * norm_b)

# Test avec des vecteurs simulés
# (En réalité, ces embeddings seraient appris)

# Simuler des embeddings de mots
embeddings = {
    'roi': np.array([0.8, 0.6, 0.2, -0.1]),
    'reine': np.array([0.75, 0.65, 0.15, -0.05]),  # Proche de roi
    'homme': np.array([0.5, 0.3, 0.4, 0.1]),
    'femme': np.array([0.45, 0.35, 0.38, 0.12]),
    'pomme': np.array([-0.2, -0.8, 0.5, 0.9]),     # Très différent
}

print("🔍 Similarités Cosinus:\n")

pairs = [
    ('roi', 'reine'),
    ('roi', 'homme'),
    ('homme', 'femme'),
    ('roi', 'pomme'),
    ('reine', 'pomme'),
]

for word1, word2 in pairs:
    sim = cosine_similarity(embeddings[word1], embeddings[word2])
    bar = '█' * int(sim * 20) if sim > 0 else ''
    print(f"{word1:<10} ↔ {word2:<10} : {sim:+.3f}  {bar}")

print("\n💡 Plus les mots sont sémantiquement proches, plus la similarité est élevée !")

---

## 🧮 Implémentation : Embedding Layer

### Principe

Un **Embedding Layer** est simplement une **lookup table** (matrice) :

```
Embedding Matrix : (vocab_size, embedding_dim)

Token ID → Ligne de la matrice
```

### Exemple

```python
vocab_size = 1000
embedding_dim = 256

# Matrice d'embeddings
E = np.random.randn(1000, 256)

# Obtenir l'embedding du token 145
token_145 = E[145]  # Vecteur de 256 dimensions
```

### Initialisation

Les embeddings sont **appris** pendant l'entraînement, mais on les initialise aléatoirement :

```python
# Initialisation petite et aléatoire
E = np.random.randn(vocab_size, embedding_dim) * 0.01
```

---

In [ ]:
class EmbeddingLayer:
    """
    Couche d'embedding simple
    """

    def __init__(self, vocab_size, embedding_dim):
        """
        Args:
            vocab_size: Taille du vocabulaire
            embedding_dim: Dimension des vecteurs d'embedding
        """
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim

        # Initialisation aléatoire petite
        self.embeddings = np.random.randn(vocab_size, embedding_dim) * 0.01

    def forward(self, token_ids):
        """
        Forward pass : token IDs → embeddings

        Args:
            token_ids: Array d'IDs de tokens
                       Shape: (batch_size,) ou (batch_size, seq_len)

        Returns:
            Embeddings correspondants
            Shape: (batch_size, embedding_dim) ou (batch_size, seq_len, embedding_dim)
        """
        return self.embeddings[token_ids]

    def backward(self, token_ids, grad_output):
        """
        Backward pass : calcule les gradients pour les embeddings

        Args:
            token_ids: IDs des tokens
            grad_output: Gradient de la loss par rapport aux embeddings

        Returns:
            Gradients pour la matrice d'embeddings
        """
        grad_embeddings = np.zeros_like(self.embeddings)

        # Accumuler les gradients pour chaque token
        # Note: plusieurs tokens peuvent partager le même ID
        np.add.at(grad_embeddings, token_ids, grad_output)

        return grad_embeddings

    def update(self, grad_embeddings, learning_rate=0.01):
        """
        Met à jour les embeddings avec gradient descent
        """
        self.embeddings -= learning_rate * grad_embeddings

print("✓ Classe EmbeddingLayer définie")

In [ ]:
# Test de l'embedding layer

vocab_size = 100
embedding_dim = 16

# Créer la couche
embed_layer = EmbeddingLayer(vocab_size, embedding_dim)

print(f"📊 Embedding Layer:")
print(f"   Vocabulaire: {vocab_size} tokens")
print(f"   Dimension: {embedding_dim}")
print(f"   Paramètres: {vocab_size * embedding_dim:,}\n")

# Test 1: Un seul token
token_id = 42
embedding = embed_layer.forward(token_id)

print(f"🔢 Token {token_id}:")
print(f"   Embedding: {embedding[:8]}... (montré 8/{embedding_dim} dims)")
print(f"   Shape: {embedding.shape}\n")

# Test 2: Séquence de tokens
sequence = np.array([5, 12, 8, 42, 7])
sequence_embeddings = embed_layer.forward(sequence)

print(f"📝 Séquence {sequence}:")
print(f"   Embeddings shape: {sequence_embeddings.shape}")
print(f"   = (seq_length={len(sequence)}, embedding_dim={embedding_dim})\n")

# Test 3: Batch de séquences
batch = np.array([
    [5, 12, 8],
    [42, 7, 15],
])

batch_embeddings = embed_layer.forward(batch)

print(f"📦 Batch {batch.shape}:")
print(f"   Embeddings shape: {batch_embeddings.shape}")
print(f"   = (batch_size={batch.shape[0]}, seq_length={batch.shape[1]}, embedding_dim={embedding_dim})")

---

## 🔍 Word2Vec : Comment Apprendre des Embeddings

### Le Problème

Comment apprendre des embeddings qui capturent la sémantique ?

### La Solution : Word2Vec (2013)

**Idée** : "You shall know a word by the company it keeps"

→ Les mots qui apparaissent dans des contextes similaires ont des sens similaires

### Deux Architectures

#### 1️⃣ **Skip-Gram** : Prédire le contexte à partir du mot

```
Phrase : "Le chat mange une souris"

Tâche : Étant donné "chat", prédire ["Le", "mange"]

Input:  "chat"
Output: "Le", "mange" (mots autour)
```

#### 2️⃣ **CBOW** (Continuous Bag of Words) : Inverse

```
Tâche : Étant donné ["Le", "mange"], prédire "chat"

Input:  "Le", "mange" (contexte)
Output: "chat" (mot central)
```

### Entraînement

En forçant le modèle à prédire les contextes, les embeddings **apprennent** la sémantique !

```python
# Après entraînement Word2Vec

# Similarités apprises
most_similar("roi") → ["reine", "prince", "monarque", ...]
most_similar("Paris") → ["France", "Londres", "capitale", ...]

# Analogies célèbres !
king - man + woman ≈ queen
Paris - France + Italy ≈ Rome
```

---

## 🎯 Embeddings dans les LLMs

### GPT vs Word2Vec

| Aspect | Word2Vec | GPT |
|--------|----------|-----|
| **Objectif** | Apprendre embeddings fixes | Génération de texte |
| **Embeddings** | Statiques (1 vecteur/mot) | Contextuels (varient selon contexte) |
| **Entraînement** | Prédiction de contexte | Next token prediction |
| **Usage** | Features pour autres tâches | Modèle complet |

### Embeddings Contextuels

Dans GPT, l'embedding **change** selon le contexte :

```python
# Phrase 1
"La banque est fermée"  # banque = institution financière
embedding_banque_1 = [0.5, -0.2, 0.8, ...]

# Phrase 2
"Je m'assois sur la banque"  # banque = siège
embedding_banque_2 = [-0.3, 0.7, -0.1, ...]  # Différent !
```

**Comment ?** Grâce au **mécanisme d'attention** (prochain notebook !) 🎯

---

## 📊 Visualisation d'Embeddings

### Problème : 256 dimensions

On ne peut pas visualiser 256 dimensions ! 😅

### Solution : Réduction de dimensionnalité

- **PCA** : Projection linéaire en 2D/3D
- **t-SNE** : Préserve les distances locales (meilleur pour viz)

```
256D → PCA/t-SNE → 2D → On peut visualiser !
```

In [ ]:
# Créer des embeddings simulés pour des mots
# (En réalité, on utiliserait des vrais embeddings pré-entraînés)

np.random.seed(42)

# Catégories de mots
words = {
    # Animaux
    'chat': np.array([0.8, 0.6, 0.2, -0.1, 0.5, 0.3, -0.2, 0.4]),
    'chien': np.array([0.75, 0.65, 0.15, -0.05, 0.45, 0.35, -0.15, 0.35]),
    'souris': np.array([0.7, 0.5, 0.25, -0.15, 0.4, 0.25, -0.25, 0.45]),
    
    # Royauté
    'roi': np.array([0.1, 0.2, 0.8, 0.7, -0.3, -0.4, 0.6, 0.5]),
    'reine': np.array([0.15, 0.25, 0.75, 0.65, -0.25, -0.35, 0.65, 0.55]),
    'prince': np.array([0.05, 0.15, 0.85, 0.75, -0.35, -0.45, 0.55, 0.45]),
    
    # Fruits
    'pomme': np.array([-0.5, -0.8, 0.1, 0.2, 0.7, 0.9, -0.6, -0.4]),
    'orange': np.array([-0.45, -0.75, 0.15, 0.25, 0.65, 0.85, -0.55, -0.35]),
    'banane': np.array([-0.55, -0.85, 0.05, 0.15, 0.75, 0.95, -0.65, -0.45]),
}

# Convertir en matrice
word_list = list(words.keys())
embeddings_matrix = np.array([words[w] for w in word_list])

print(f"📊 Embeddings créés:")
print(f"   {len(word_list)} mots")
print(f"   {embeddings_matrix.shape[1]} dimensions")

# Réduire à 2D avec PCA
pca = PCA(n_components=2)
embeddings_2d = pca.fit_transform(embeddings_matrix)

print(f"\n✓ Réduction PCA: {embeddings_matrix.shape[1]}D → 2D")

In [ ]:
# Visualiser les embeddings
fig, ax = plt.subplots(figsize=(12, 10))

# Couleurs par catégorie
categories = {
    'animaux': ['chat', 'chien', 'souris'],
    'royauté': ['roi', 'reine', 'prince'],
    'fruits': ['pomme', 'orange', 'banane'],
}

colors = {
    'animaux': 'steelblue',
    'royauté': 'purple',
    'fruits': 'orange',
}

# Plot par catégorie
for category, word_group in categories.items():
    indices = [word_list.index(w) for w in word_group]
    points = embeddings_2d[indices]
    
    ax.scatter(points[:, 0], points[:, 1], 
              s=200, c=colors[category], alpha=0.6, 
              edgecolors='black', linewidth=2,
              label=category.capitalize())
    
    # Annoter
    for i, word in enumerate(word_group):
        ax.annotate(word, (points[i, 0], points[i, 1]),
                   xytext=(5, 5), textcoords='offset points',
                   fontsize=12, fontweight='bold')

ax.set_xlabel('Composante Principale 1', fontsize=12, fontweight='bold')
ax.set_ylabel('Composante Principale 2', fontsize=12, fontweight='bold')
ax.set_title('Visualisation des Embeddings (PCA 2D)', fontsize=14, fontweight='bold')
ax.legend(loc='best', fontsize=12)
ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 Observation:")
print("   - Les mots de la même catégorie sont PROCHES dans l'espace")
print("   - Les catégories différentes sont ÉLOIGNÉES")
print("   - C'est la MAGIE des embeddings ! 🎨")

---

## 🎓 Résumé et Concepts Clés

### Ce qu'on a appris

✅ **Problème des IDs** : Ne capturent pas la sémantique  
✅ **Embeddings** : Vecteurs denses qui capturent le sens  
✅ **Similarité** : Mots similaires = vecteurs proches  
✅ **Embedding Layer** : Lookup table apprise  
✅ **Word2Vec** : Comment apprendre des embeddings  

### Concepts Clés

1. 🔑 **Dense Embedding** : Représentation compacte et riche
2. 🔑 **Cosine Similarity** : Mesure de proximité sémantique
3. 🔑 **Embedding Matrix** : (vocab_size, embedding_dim)
4. 🔑 **Word2Vec** : Skip-gram et CBOW
5. 🔑 **Contextuels** : GPT adapte les embeddings au contexte

### Formules Importantes

```python
# Forward (lookup)
embedding = E[token_id]  # Simple indexation !

# Similarité
sim = (A · B) / (||A|| × ||B||)

# Dimension
Embedding matrix : (vocab_size, embedding_dim)
GPT-2 : (50257, 768)
GPT-3 : (50257, 12288)
```

---

## 🚀 Prochaine Étape

**Notebook 03 - Attention Mechanism** 🎯

Maintenant qu'on a des embeddings, on va apprendre le **mécanisme d'attention** !

```
Texte → Tokens → IDs → Embeddings → ATTENTION → Contexte enrichi
                                     ↑ Prochain !
```

**Pourquoi c'est important ?**
- L'attention est le **cœur** du Transformer
- Permet de capturer les **relations** entre tous les tokens
- C'est ce qui rend les LLMs si puissants

**Teaser** :
```python
# L'attention va répondre à des questions comme:
# Dans "Le chat de Marie est mignon. Il aime jouer."
# "Il" fait référence à quoi ? → "chat" (attention élevée)
```

---

## 🎯 Exercices (Optionnels)

1. **Implémenter** : Ajouter des méthodes `most_similar()` à l'EmbeddingLayer
2. **Visualiser** : Utiliser t-SNE au lieu de PCA et comparer
3. **Expérimenter** : Créer des embeddings pour 50 mots et visualiser
4. **Analogies** : Implémenter `king - man + woman = queen`
5. **Comparer** : Charger de vrais embeddings Word2Vec ou GloVe et explorer

---

**Prêt pour l'attention ? C'est le concept le plus important ! 🚀**

**→ Notebook suivant : `03_attention_mechanism.ipynb`**